#### GISC 420 T1 2022
# A simple example getting Python to drive JavaScript
A lot of the nicest new frameworks (for mapping or anything else) happen first these days, or develop most quickly in the web-space. This has a great deal to do with the enormous resources poured into the web as a platform, which have seen previously unimaginable development of the JavaScript (JS) prgramming language and of the associated virtual machines (which are everywhere now... every phone, every computer, and so on...).

As a result anyone writing code today should at least be web-aware. And that means being somewhat acquainted with the JS world.

We'll look at an example of web-programming (non-map related) in class, but to also give you a sense of why this is an exciting space below we use [`pydeck`](https://pydeck.gl/index.html) to drive the [`deck.gl`](https://deck.gl/) framework for interactive 3D visualisation.

## Getting set up
To run this notebook, you'll need to first download [the zip file for this material](https://github.com/DOSull/GISC-420/blob/master/labs/deck/deck.zip?raw=true), which includes this notebook and also the data. 

Then you'll need to make a new environment. These instructions are from [here](https://pydeck.gl/installation.html). 

+ Open the **Anaconda console** and make a new environment as follows:
    
    `conda create -n deck -c conda-forge pydeck jupyter`

+ Next start the environment:

    `conda activate deck`

+ Then issue these two commands:

    `jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck`
    
    `jupyter nbextension enable --sys-prefix --py pydeck`

That's it. You should be able to start a notebook as usual, this time in the `deck` environment, and the code below should work. Note that the example may not work in VSCode---I think due to security settings of the HTML viewer (it's not insecure... all the data are local, it's some setting associated with 'cross-origin' resources).

## Making a `pydeck` 'app'
These data are briefly described on [this page](https://github.com/DOSull/Geog315/tree/master/labs/mini-project/anz-placenames) where I note the use of the [nga kupu](https://github.com/TeHikuMedia/nga-kupu) tools to classify a collection of Aotearoa New Zealand placenames as consisting of parts (i.e. words) that are either clearly te reo, clearly not, or maybe te reo. Each placename has a count of how many of its constituent words falls into each of these categories.

For `deck.gl` as for many JS applications, the preferred file format is [JSON](https://en.wikipedia.org/wiki/JSON) (JavaScript Object Notation), so I made a version of the data in that format. (I used the `jsonlite` package in *R* for this, but there are Python libraries for working with JSON also available, such as the standard library [`json`](https://docs.python.org/3.9/library/json.html)).

Anyway, to start we need to import `pydeck`:

In [1]:
import pydeck
import string

Then we need to specify some setups. 

+ `DATA_URL` is where we have the data (this should be in the folder downloaded with in the zip file).
+ `INITIAL_VIEW_STATE` builds a `ViewState` object specifying the initial view location, direction, angle and so on for the deck application.
+ `names` is a `Layer` object, which specifies, a data source (we pass `DATA_URL` as a parameter, and a series of settings pertaining to how this layer will be displayed.

THe details of a the [**TextLayer**] type are available in the documentation for the JS API [here](https://deck.gl/docs/api-reference/layers/text-layer). Partial details of the `pydeck` 'wrapper' for this are available [here](https://pydeck.gl/gallery/text_layer.html). You should be able to see similarities between the Python code and the JS code, but there are differences in how parameters are named, for example. Where Python uses `snake_case`, JS uses `camelCase`. 

Also things like how expressions to determine `get_color` are specified look pretty strange. We are essentially passing a piece JS code as a string to the `Layer` object constructor, so that it can be parsed into proper JS when it makes the output. 

In [2]:
DATA_URL = "tnames.json"
chars = string.ascii_letters + "āēīōūĀĒŌĪŪ1234567890/ "

INITIAL_VIEW_STATE = pydeck.ViewState(
    latitude = -41,
    longitude = 174,
    zoom = 5,
    max_zoom = 20,
    pitch = 0,
    bearing = 0
)

names = pydeck.Layer(
    'TextLayer',
    DATA_URL,
    opacity = 0.65,
    get_text = "name",
    size_units = "meters",
    get_size = 1000,
    size_max_pixels = 32,
    get_position = ["lng", "lat"],
    get_color = 'reo > 0 ? [204, 0, 0] : [0, 0, 204]',
    billboard = True,
    character_set = pydeck.types.String(chars),
    font_family = pydeck.types.String("Times, serif"),
    pickable = True
)

And now we can make the deck app, by passing all this information into the `pydeck.Deck` constructor. The additional information here specifies the preferred basemap style, per the [options listed here](https://pydeck.gl/deck.html).

When you run the code below it will open an interactive output cell, and you can zoom and navigate around in it within this notebook

In [3]:
app = pydeck.Deck(
    initial_view_state = INITIAL_VIEW_STATE,
    map_style = "light_no_labels",
    layers = [names],
)
app.to_html("tereo-names.html", iframe_height = 800)

Because we are running this in a notebook we just get a locally viewable web page. If we run the code as a script we can also save to a web page.

I suggest experimenting with other options in the `pydeck.Layer()` constructor function. Also make sure to take a look at the generated HTML code (it's surprisingly simple, but it won't open in a browser, because we generated it in a notebook).

Try visualising a different dataset, using a different deck layer format. For example, you can take a look at something simple like the California counties data from weeks ago, using the [GeoJsonLayer](https://deck.gl/docs/api-reference/layers/geojson-layer). You'll need to convert those files to GeoJSON format and also latitude-longitude format (EPSG code 4326).

### Some more examples
And [here's one I prepared earlier](https://dosull.github.io/commute-viewer/commute-viewer-app/). This includes hand-coded JS to implement a custom interface.

And [here's another](https://dosull.github.io/what3chords/what3chords-app/) (**turn down your sound** is all I'm saying).

## If you found this fun...

This is pretty cool stuff.

You can use [`kepler.gl`](https://kepler.gl/) to [make simple similar apps](https://southosullivan.com/misc/loi.html) without any code at all.

If you have to make a custom user interface, then there's no real way to avoid... web development... and there aren't enough hours in the month to cover that in the context of this course. 

Suffice to say, web development is a field worth investing some time and energy in, especially if your interests lie in the direction of high interaction graphical applications.